In [1]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json
import wget
import os

In [2]:
# def scroll(driver, timeout):
#     scroll_pause_time = timeout

#     # Get scroll height
#     last_height = driver.execute_script("return document.body.scrollHeight")

#     while True:
#         # Scroll down to bottom
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#         # Wait to load page
#         time.sleep(scroll_pause_time)

#         # Calculate new scroll height and compare with last scroll height
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             # If heights are the same it will exit the function
#             break
#         last_height = new_height


def all_links(url):
    driver = webdriver.Chrome('chromedriver.exe')
    # implicitly_wait tells the driver to wait before throwing an exception
    driver.implicitly_wait(100)
    
    driver.get(url)

    for j in range(0, 50):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

    # Once scroll returns bs4 parsers the page_source
    soup_a = BeautifulSoup(driver.page_source, 'lxml')
    # Them we close the driver as soup_a is storing the page source
    driver.close()

    links=[]
    body = soup_a.find('body')
    script = body.find('script', text=lambda t: t.startswith('window._sharedData'))
    page_json = script.string.split(' = ', 1)[1].rstrip(';')
    data = json.loads(page_json)
    for link in data['entry_data']['TagPage'][0]['graphql']['hashtag']['edge_hashtag_to_media']['edges']:
        links.append('https://www.instagram.com'+'/p/'+link['node']['shortcode']+'/')

    return links

In [3]:
hashtag = 'vietnamesecommunity'
links = all_links('https://www.instagram.com/explore/tags/'+hashtag)

In [5]:
# specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome()

#open the webpage
driver.get("http://www.instagram.com")

#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

#enter username and password
username.clear()
username.send_keys("XXXXXXXXXX")
password.clear()
password.send_keys("XXXXXX")

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

#We are logged in!

In [6]:
time.sleep(5)
alert = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click()
#alert2 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click()

In [7]:
images = []

#follow each image link and extract only image at index=1
for a in links:
    driver.get(a)
    time.sleep(5)
    img = driver.find_elements_by_tag_name('img')
    img = [i.get_attribute('src') for i in img]
    images.append(img[1])

In [8]:
#path = os.getcwd()
path = r"C:\Users\samsu\Desktop\Spring21\DAEN 690\Code"
path = os.path.join(path, hashtag)

#create the directory
os.mkdir(path)
path

'C:\\Users\\samsu\\Desktop\\Spring21\\DAEN 690\\Code\\vietnamesecommunity'

In [9]:
images = list(filter(None,images))
#download images
counter = 0
for image in images:
    save_as = os.path.join(path, hashtag + str(counter) + '.jpg')
    wget.download(image, save_as)
    counter += 1

100% [..............................................................................] 6475 / 6475

In [10]:
driver.close()